<a href="https://colab.research.google.com/github/dorisratego/TensorFlowML-AI/blob/main/ParallelizingETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import multiprocessing

In [9]:
#Get dataset
train_data = tfds.load('cats_vs_dogs', split='train', with_info=True)

In [10]:
#create a list of these files and use tf.Data.Dataset.list_files to load them
file_pattern = f'/root/tensorflow_datasets/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord*'
files = tf.data.Dataset.list_files(file_pattern)

In [11]:
#load files into a dataset using files.interleave
train_dataset = files.interleave(
                     tf.data.TFRecordDataset, 
                     cycle_length=4, #specify the number of parallel calls to execute
                     num_parallel_calls=tf.data.experimental.AUTOTUNE # number of parallel calls based on number of CPU
                )

In [12]:
#create the mapping function that loads the raw TFRecord and converts it to usable content
def read_tfrecord(serialized_example):
  feature_description={
      "image": tf.io.FixedLenFeature((), tf.string, ""),
      "label": tf.io.FixedLenFeature((), tf.int64, -1),
  }
  example = tf.io.parse_single_example(
       serialized_example, feature_description
  )
  image = tf.io.decode_jpeg(example['image'], channels=3)
  image = tf.cast(image, tf.float32)
  image = image / 255
  image = tf.image.resize(image, (300,300))
  return image, example['label']


In [13]:
#calling the mapping function to parallelize the transformation of data
cores = multiprocessing.cpu_count()
print(cores)
train_dataset = train_dataset.map(read_tfrecord, num_parallel_calls=cores)
train_dataset = train_dataset.cache() #cache the dataset in memory.

2


In [14]:
#parallelizing Loading and training
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [15]:
#train model
model.fit(train_dataset, epochs=10, verbose=1)


NameError: ignored